<a href="https://colab.research.google.com/github/deekshakoul/Examples-of-DL-NLP-using-Pytorch/blob/master/Sentiment_Analysis_using_LSTM_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

https://towardsdatascience.com/sentiment-analysis-using-lstm-step-by-step-50d074f09948 

https://github.com/prakashpandey9/Text-Classification-Pytorch/blob/master/load_data.py 

https://www.analyticsvidhya.com/blog/2020/01/first-text-classification-in-pytorch/ 

https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

In [ ]:
df = pd.read_csv('/content/drive/My Drive/iisc/summer/datasets/train.csv')
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows
df_test = pd.read_csv('/content/drive/My Drive/iisc/summer/datasets/test.csv')
df_test = df_test.sample(frac=1).reset_index(drop=True) #shuffle rows

Pytorch - Text Classification

---

* Handling OOV words - pytorch supports feature that replaces the rare words in our training data with Unknown token.

* Handling Variable Length sequences - As we deal with static networks(models), so we convert the variable length  input sentences into sentences with the same length(threshod also called as sequence length) by adding padding tokens.
The issue with padding tokens is that now our network also tries to model the padding like as any other information.
This is taken care by **Pytorch's Packed Padding sequence** 

Packed padding ignores the input timesteps with padding token. These values are never shown to the Recurrent Neural Network which helps us in building a dynamic Recurrent Neural Network.


![alt text](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/01/Untitled-Diagram1.png)



In [ ]:
#deal with tensors
import torch   

#handling text data
from torchtext import data  

#Reproducing same results
SEED = 147

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

Pre-processing

In [ ]:
#Punctuations
import string
sym = list(string.punctuation)
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords")

stopwords = set(stopwords.words("english")) | set(["br"]) | set(sym)  | set(["/><br",'\'s'])
#Tokens to discard during the preprocessing step

In [ ]:
TEXT = data.Field(tokenize='spacy',lower=True,batch_first=True,include_lengths=True,stop_words=stopwords)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
fields = [('label', LABEL), ('text',TEXT)]

In [ ]:
train_data = data.TabularDataset(skip_header = True,path='/content/drive/My Drive/iisc/summer/datasets/train.csv',format = 'csv', fields=fields)#defines data stored in csv in terms of column
#torchtext.data.dataset.TabularDataset
test_data = data.TabularDataset(skip_header = True,path='/content/drive/My Drive/iisc/summer/datasets/test.csv',format = 'csv', fields=fields)#defines data stored in csv in terms of column

In [ ]:
print(vars(train_data.examples[1])) #Defines a single training or test example

{'label': '0', 'text': ['wanted', 'see', 'movie', 'article', 'film', 'magazine', "n't", 'highly', 'recommended', 'one', 'critic', 'storyline', 'different', 'sure', 'could', 'good', 'movie', 'right', 'hands', 'directing', 'acting', 'awful', 'feeling', 'watching', 'movie', 'made', 'bunch', 'amateurs', 'although', 'movie', 'started', 'promisingly', 'got', 'worse', 'worse', 'think', 'unoriginal', 'movie', 'awkward', 'characters', '..', 'still', 'think', 'worth', 'watching', "n't", 'seen', 'films', 'subjecting', 'gay', 'porn', "n't", 'keep', 'expectations', 'high', 'though', 'disappointed']}


Creating Vocab and indexing words while taking care of above said problems of OOV words and  variable length sequence

In [ ]:
#Initializing Glove embeddings

TEXT.build_vocab(train_data, min_freq=3, vectors = "glove.6B.100d") # Vocab object from one or more datasets.
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 398209/400000 [00:17<00:00, 23983.54it/s]

In [ ]:
#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

Size of TEXT vocabulary: 41753
Size of LABEL vocabulary: 2
[('movie', 43063), ('film', 39285), ("n't", 32846), ('one', 26135), ('like', 20100), ('good', 14905), ('would', 13458), ('even', 12429), ('time', 12338), ('story', 11725)]
defaultdict(<function _default_unk_index at 0x7fd6d8fc5d90>, {'<unk>': 0, '<pad>': 1, 'movie': 2, 'film': 3, "n't": 4, 'one': 5, 'like': 6, 'good': 7, 'would': 8, 'even': 9, 'time': 10, 'story': 11, 'really': 12, 'see': 13, 'well': 14, '...': 15, 'much': 16, 'could': 17, 'get': 18, 'people': 19, 'bad': 20, 'great': 21, 'also': 22, 'first': 23, 'made': 24, 'make': 25, 'way': 26, 'movies': 27, '/>the': 28, 'think': 29, 'characters': 30, 'character': 31, 'two': 32, 'watch': 33, 'films': 34, 'many': 35, 'seen': 36, 'never': 37, 'plot': 38, 'life': 39, 'little': 40, 'love': 41, 'acting': 42, 'best': 43, 'show': 44, 'know': 45, 'ever': 46, 'man': 47, 'better': 48, 'still': 49, 'end': 50, 'say': 51, 'scene': 52, 'scenes': 53, 'go': 54, 'something': 55, "'ve": 56, 'b

Iterator over batch sizes

In [ ]:
#A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = 64,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True)#,
#    device = device)

#or use a simply torchtext.data.Iterator(dataset, batch_size, sort_key=None, device=None, batch_size_fn=None, 
#train=True, repeat=False, shuffle=None, sort=None, sort_within_batch=None)
#https://torchtext.readthedocs.io/en/latest/data.html#torchtext.data.Iterator 

#can also use torch.utils.data.DataLoader ??

**torch.nn.utils.rnn.pack_padded_sequence**(input, lengths, batch_first=False, enforce_sorted=True)

* Packs a Tensor containing padded sequences of variable length.
* Input can be of size T x B x * where T is the length of the longest sequence 
* If batch_first is True, B x T x * input is expected.
* Parameters:
    * input (Tensor) – padded batch of variable length sequences
    * lengths (Tensor) – list of sequences lengths of each batch element.
    * batch_first
    * enforce_sorted (bool, optional) – if True, the input is expected to contain sequences sorted by length in a decreasing order. If False, the input will get sorted unconditionally. Default: True.
* Returns a PackedSequence object    
* Internally packed sequence is a tuple of two lists. One contains the elements of sequences. Elements are interleaved by time steps

In [ ]:
#example to understand packed sequence better
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
c = torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])
#Data part is just all the tensors concatenated along the time axis. Batch_size is actually the array of batch sizes at each time step
#The batch_sizes=[2, 2, 1] represents grouping [1, 3] [2, 4] and [3] respectively(according to time step)

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=None, unsorted_indices=None)

 **Modellling NN**
 
 Two functions defined as:
 * init : arguments passed to the class are initialized by this constructor
 * forward : defines forward pass of the inputs.

**Layers of Model:**
* Embedding layer : Get word embeddings for each indexed word, pytorch has nn.embedding which takes input as: 
    * num_embeddings: No. of unique words in dictionary
    * embedding_dim:  No. of dimensions for representing a word (user-defined)

* LSTM:
Detail explanation of paramaters, input, output of lstm can be found on this stackoverflow [SO link](https://stackoverflow.com/questions/45022734/understanding-a-simple-lstm-pytorch)
 
* Linear Layer
* Pack Padding: As explained above, it defines a dynamic recurrent neural network.  It simply ignores the values and returns the hidden state of the non padded element.




In [57]:
class LSTMNetwork(nn.Module):
  #input_size: size of input per time step, in this case size of xt
  def __init__(self, output_size, vocab_size, embedding_size, hidden_size, nlayers):
    super(LSTMNetwork,self).__init__();
    #embedding 
    #returns a matrix of size vocab_size x embedding_size
    self.embeds =  nn.Embedding(vocab_size, embedding_size)  


    self.lstm = nn.LSTM(embedding_size, 
                        hidden_size, 
                        num_layers=nlayers, 
                        batch_first=True);
   
    #fully connected layer
    self.fc = nn.Linear(hidden_size, output_size)
    #sigmoid layer
    self.sigmoid = nn.Sigmoid()

  def forward(self, text, text_lengths):
    #text = [batch size,sent_length]
    # embedded shape = (batch_size, sentence_length,  embedding_size)
    embedded = self.embeds(text) 
    
    #using packed packing sequence
    embedding_packed = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, batch_first=True)# If batch_first is True, B x T x * input is expected. 

    #input, (h_0, c_0) where input is of shape (seq_len, batch, input_size) or  also be a packed variable length sequenc
    #h_0 of shape (num_layers * num_directions, batch, hidden_size)
    #c_0 of shape (num_layers * num_directions, batch, hidden_size)
    #no need to provide as by default they are initialized as zero 
    packed_output, (final_hidden_state, final_cell_state)  = self.lstm(embedding_packed)
    #output, (h_n, c_n): output of shape (seq_len, batch, num_directions * hidden_size) or packedSequence object
    #h_n of shape (num_layers * num_directions, batch, hidden_size)  : hidden state when  t = seq_len
    #c_n of shape (num_layers * num_directions, batch, hidden_size)  : cell state when  t = seq_len
        # final_hidden_state is 1 x batch_size x hidden_size
        # [ [h1,h2,h3], [h1,h2,h3]  ] for two reviews, batch = 2, so FINAL hidden state size is 1 x 2 x 3 
        # therefore doing final_hidden_state[-1] will give dimesnions as 2x3 and thus can now be easily passed to linear function(fc) as it takes only 2D tensors 
    
    out_fc = self.fc(final_hidden_state[-1]) 
    #out_fc  will be (batch_size, output_size) : this is similar to FNN
    out = self.sigmoid(out_fc);
    return out ; 

**Instantiate the model class**

In [58]:
output_size = 1
vocab_size = len(TEXT.vocab)
embedding_size = 100 #this is TEXT.vocab.vectors.shape
hidden_size = 200
nlayers = 1
model = LSTMNetwork(output_size, vocab_size, embedding_size, hidden_size, nlayers)

In [59]:
print(model)

LSTMNetwork(
  (embeds): Embedding(41753, 100)
  (lstm): LSTM(100, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


**Pretrained word embeddings**

vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.

In [60]:
pretrained_embeddings = TEXT.vocab.vectors
model.embeds.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape) # each words is rep by 100 size vector

torch.Size([41753, 100])


**Instantiate loss and optimizer Class**

Here I have used BCE loss as output size of my last linear layer is kept as 1.

I could have used CrossEntropyLoss with output size of LL as 2, the CEloss itself calculates softmax and corresponding label

In [61]:
criterion = nn.BCELoss()
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

**TRAINING PHASE**

In [64]:
#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def trainingNetwork(model, iterator, optimizer, criterion):
  epoch_loss = 0.0
  epoch_accuracy = 0.0
  
  for batch in iterator:
    texts, text_lengths = batch.text # we have set inlude_lengths as True while defining data.TEXT
    optimizer.zero_grad() 
    predictions_batch  =  model(texts, text_lengths).squeeze()

    #compute the loss
    loss = criterion(predictions_batch, batch.label)
    acc = binary_accuracy(predictions_batch, batch.label)

    #backpropage the loss and compute the gradients
    loss.backward()       
        
    nn.utils.clip_grad_norm_(model.parameters(), clip);

    #update the weights
    optimizer.step()   
    #loss and accuracy
    epoch_loss += loss.item()  
    epoch_accuracy += acc.item()     

  return epoch_loss / len(iterator), epoch_accuracy / len(iterator)  

In [ ]:
#train_iterator - BucketIterator 
N_EPOCHS = 5
best_valid_loss = float('inf')
clip=5
for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = trainingNetwork(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    '''valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')'''
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')